In [10]:
import os
import pandas as pd
import numpy as np
import boto3 

import sagemaker
from sagemaker import Session  
from sagemaker import get_execution_role
from sagemaker.s3 import S3Uploader

#library for data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

#library for Pytorch
from sagemaker.pytorch import PyTorch

#library for Bias and Explainability
from sagemaker import clarify 

#library for differential privacy
from opacus import PrivacyEngine

#library for carbon emission
from codecarbon import EmissionsTracker

In [11]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name
s3_client = boto3.client("s3") 

region: us-east-1
role: arn:aws:iam::247385444016:role/service-role/AmazonSageMaker-ExecutionRole-20210426T062919
input_data_path: s3://responsibleai/data


In [ ]:
data_path = 's3://responsibleai/churn_prediction/data'
output_path = 's3://responsibleai/churn_prediction/output'

In [ ]:
#Start Emissions Tracker
tracker = EmissionsTracker(project_name = "churn_prediction",
                           output_dir = "../output/",
                           measure_power_secs = 15,
                           save_to_file = True)

### Train the Model

In [15]:
pytorch_estimator = PyTorch(entry_point = '../code/train.py',
                            source_dir = '../code'
                            instance_type='ml.p3.2xlarge',
                            role = role,
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            hyperparameters = {'epochs': 100, 
                                               'batch-size': 64, 
                                               'learning-rate': 0.003})

In [ ]:
pytorch_estimator.fit({'train': 's3://responsibleai/data/churn.csv'})

2022-02-28 18:36:06 Starting - Starting the training job...
2022-02-28 18:36:30 Starting - Launching requested ML instancesProfilerReport-1646073366: InProgress
......

### Calculate Emissions

In [ ]:
#stop Emissions Tracker and load file
emissions: float = tracker.stop()
S3Uploader.upload("../output/emissions.csv", 's3://responsibleai/churn_prediction/output/emissions.csv')
print(emissions)

### Deploy Model / Create Model

In [ ]:
## Create the Model  - TODO
model_name = "churn-clarify-model" 
predictor = pytorch_estimator.deploy(initial_instance_count=1, 
                         instance_type='ml.p3.2xlarge',
                         model_name = "churn-clarify-model")

### Set up Sagemaker Clarify

In [ ]:
## set the Clarify processor
clarify_processor = clarify.SageMakerClarifyProcessor( 
                        role=role, 
                        instance_count=1, 
                        instance_type="ml.m5.xlarge", 
                        sagemaker_session=session) 

## point to the data files - TODO
bias_report_output_path = 's3://responsibleai/bias'

df = pd.read_csv("../data/churn.csv")
df = df.drop(["Surname", "CustomerId", "RowNumber"], axis=1)
categorical = ["Geography", "Gender"]
churn_frame = pd.get_dummies(df, prefix = categorical)
X = churn_frame.drop(['Exited'], axis=1)

scaler = StandardScaler()
X_array  = scaler.fit_transform(X)
X = pd.DataFrame(X_array)

X.to_csv("../data/test.csv", index=False, header=False)

train_uri = S3Uploader.upload("../data/test.csv", 's3://responsibleai/data/test.csv')

### Model Explainability

In [ ]:
#calculate explainability
shap_config = clarify.SHAPConfig( 
    num_samples=15, 
    agg_method="mean_abs", 
    save_local_shap_values=True,) 

explainability_data_config = clarify.DataConfig( 
    s3_data_input_path=train_uri, 
    s3_output_path=bias_report_output_path, 
    label="Exited", 
    headers=churn_frame.columns.to_list(), 
    dataset_type="text/csv") 

model_config = clarify.ModelConfig( 
    model_name=model_name, 
    instance_type="ml.p3.2xlarge", 
    instance_count=1,
    accept_type="text/csv", 
    content_type="text/csv",) 

clarify_processor.run_explainability( 
    data_config=explainability_data_config, 
    model_config=model_config, 
    explainability_config=shap_config,) 


### Model Bias

In [ ]:
bias_data_config = clarify.DataConfig( 
    s3_data_input_path=train_uri, 
    s3_output_path=bias_report_output_path, 
    label="Exited", 
    headers=churn_train.columns.to_list(), 
    dataset_type="text/csv") 

model_config = clarify.ModelConfig( 
    model_name=model_name, 
    instance_type="ml.m5.xlarge", 
    instance_count=1,
    accept_type="text/csv", 
    content_type="text/csv",) 

predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

bias_config = clarify.BiasConfig( 
    label_values_or_threshold=[1], 
    facet_name="Gender", 
    facet_values_or_threshold=[0]) 

clarify_processor.run_bias( 
    data_config=bias_data_config, 
    bias_config=bias_config, 
    model_config=model_config, 
    model_predicted_label_config=predictions_config, 
    pre_training_methods="all", 
    post_training_methods="all") 